In [45]:
# pip install prophet == 1.1.6
# pip install keras == 3.9.0 tensorflow == 2.19.0
# pip install neuralprophet == 0.8.0
# pytorch-forecasting-1.3.0
#pip install nixtla == 0.6.6
#pip install pmdarima == 2.0.4
#pip install scikeras == 0.13.0
# !pip install keras-tcn==3.1.2

In [21]:
from sklearn.model_selection import PredefinedSplit, GridSearchCV
import xgboost as xgb

In [22]:
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#Check if Y is stationary
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error
from math import sqrt

import statsmodels.api as sm

import itertools
from itertools import combinations
from scipy.stats import spearmanr
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

# Modeling Pipeline (Feature selection, Scaling, Model testing)
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    mean_squared_error, 
    r2_score, 
    mean_absolute_percentage_error,
    mean_absolute_error,
    root_mean_squared_error  # New recommended way for RMSE
)

from xgboost import XGBRegressor
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
from neuralprophet import NeuralProphet
import xgboost as xgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

import tensorflow as tf
from pytorch_forecasting.models import TemporalFusionTransformer, NBeats
from pytorch_forecasting.data.timeseries import TimeSeriesDataSet
import torch


from sklearn.model_selection import TimeSeriesSplit, PredefinedSplit

from nixtla import NixtlaClient
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from scikeras.wrappers import KerasRegressor
from keras.optimizers import Adam
from joblib import Parallel, delayed

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tcn import TCN

In [23]:
# Set pandas display options
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)       # Set display width
pd.set_option('display.max_colwidth', 100) # Show full feature lists
pd.set_option('display.float_format', '{:.4f}'.format)  # 4 decimal places

# If you want to force standard notation (no scientific):
pd.set_option('display.float_format', lambda x: '%.4f' % x if abs(x) > 1e-4 else '%.4e' % x)

In [24]:
def load_dfs_from_folder(folder_path):
    """Loads DataFrames from files in a specified folder and returns a dictionary."""
    dfs = {}
    # List all files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".pkl"):
            key = file_name.replace(".pkl", "")  # Extract key from the file name
            file_path = os.path.join(folder_path, file_name)
            
            # Load the dataframe from the pickle file
            with open(file_path, 'rb') as f:
                dfs[key] = pickle.load(f)
            print(f"Loaded {key} from {file_path}")
    
    return dfs

# Load both product_dfs and lagged_product_dfs from their respective folders
product_dfs = load_dfs_from_folder("product_dfs_folder")
lagged_product_dfs = load_dfs_from_folder("lagged_product_dfs_folder")

Loaded P1 from product_dfs_folder\P1.pkl
Loaded P11 from product_dfs_folder\P11.pkl
Loaded P12 from product_dfs_folder\P12.pkl
Loaded P13 from product_dfs_folder\P13.pkl
Loaded P14 from product_dfs_folder\P14.pkl
Loaded P16 from product_dfs_folder\P16.pkl
Loaded P20 from product_dfs_folder\P20.pkl
Loaded P3 from product_dfs_folder\P3.pkl
Loaded P36 from product_dfs_folder\P36.pkl
Loaded P4 from product_dfs_folder\P4.pkl
Loaded P5 from product_dfs_folder\P5.pkl
Loaded P6 from product_dfs_folder\P6.pkl
Loaded P8 from product_dfs_folder\P8.pkl
Loaded P9 from product_dfs_folder\P9.pkl
Loaded Sales_CPI from product_dfs_folder\Sales_CPI.pkl
Loaded P1 from lagged_product_dfs_folder\P1.pkl
Loaded P11 from lagged_product_dfs_folder\P11.pkl
Loaded P12 from lagged_product_dfs_folder\P12.pkl
Loaded P13 from lagged_product_dfs_folder\P13.pkl
Loaded P16 from lagged_product_dfs_folder\P16.pkl
Loaded P20 from lagged_product_dfs_folder\P20.pkl
Loaded P3 from lagged_product_dfs_folder\P3.pkl
Loaded P36 

In [25]:
for product_id in product_dfs.keys():
    product_dfs[product_id] = product_dfs[product_id].rename(columns={product_id: "Sales"})

for product_id in lagged_product_dfs.keys():
    lagged_product_dfs[product_id] = lagged_product_dfs[product_id].rename(columns={product_id: "Sales"})

In [26]:
product_dfs['P1'].head()

,Sales,MAB_ELE_SHP840,PRI27276_org,PRO27826_org,MAB_ELE_PRO276,MAB_ELE_SHP1100
month_year,,,,,,
2018-10-01,35774028.5209,127.8088,109.1196,118.6708,124.2279,130.9893
2018-11-01,5063648.6000,117.6759,109.2248,120.4670,127.4041,132.9341
2018-12-01,37321267.9382,123.2801,109.3301,105.3787,120.5186,131.2613
2019-01-01,27090400.9380,111.0438,109.7510,107.1749,104.7763,113.0576
2019-02-01,34132093.4229,116.7369,109.8562,110.6476,109.5970,117.7047


In [27]:
lagged_product_dfs['P8'].head()

,Sales,PRI27840_org,RohCOPPER1000_org,time_idx,year,month,P8_lag_1,P8_lag_2,P8_lag_5,P8_lag_6,P8_lag_10,P8_ma_1,P8_ma_2,P8_ma_5,P8_ma_6,P8_ma_10
month_year,,,,,,,,,,,,,,,,
2019-08-01,398332.2744,110.6561,75.7745,10,2019,8,368081.2008,420287.0223,582419.0346,361474.5342,580778.2653,398332.2744,383206.7376,380139.7221,413852.9409,400303.6854
2019-09-01,1086855.2918,111.0503,76.4355,11,2019,9,398332.2744,368081.2008,416386.5006,582419.0346,518398.3785,1086855.2918,742593.7831,514233.4804,497925.6504,457149.3767
2019-10-01,112014.9227,111.0322,76.4097,12,2019,10,1086855.2918,398332.2744,297611.6126,416386.5006,267418.3494,112014.9227,599435.1073,477114.1424,447197.0541,441609.0341
2019-11-01,540208.8029,111.0354,77.7720,13,2019,11,112014.9227,1086855.2918,420287.0223,297611.6126,372627.9465,540208.8029,326111.8628,501098.4985,487629.9191,458367.1197
2019-12-01,491332.7200,111.2866,80.6535,14,2019,12,540208.8029,112014.9227,368081.2008,420287.0223,361474.5342,491332.7200,515770.7614,525748.8024,499470.8688,471352.9383


In [29]:
def preprocessing_pipeline(train, val=None, test=None, outlier_treatment=True):

    def winsorize_function(df, cols, lower_quantile=0.01, upper_quantile=0.99):
        """Apply Winsorization only to the product sales column"""
        df = df.copy()
        bounds = {}
        for col in cols:
            q1 = df[col].quantile(lower_quantile)
            q3 = df[col].quantile(upper_quantile)
            df[col] = df[col].clip(lower=q1, upper=q3)
            bounds[col] = (q1, q3)  # Store actual percentile values
            print(f"{col}: Winsorized Bounds -> Lower = {q1:.2f}, Upper = {q3:.2f}")

        return df, bounds
        
    def process_dataset(df, cols, is_train=True, bounds=None):
        df = df.copy() 
        if is_train:
            if outlier_treatment:
                df, bounds = winsorize_function(df, cols)
            else:
                bounds = {}  
        else:
            if outlier_treatment:
                if bounds is None:
                    raise ValueError("Bounds must be provided for validation and test datasets.")
                for col in cols: 
                    if col in bounds:
                        lower, upper = bounds[col]
                        df[col] = df[col].clip(lower, upper)  # Corrected clipping
        return (df, bounds) if is_train else df

    # Process the training dataset
    train, bounds = process_dataset(train, cols = train.columns, is_train=True)

    # Process validation and test datasets with correct bounds
    if val is not None:
        val = process_dataset(val, cols = val.columns, is_train=False, bounds=bounds)

    if test is not None:
        test = process_dataset(test, cols = test.columns, is_train=False, bounds=bounds)
        
    # Return the datasets 
    if test is not None and val is not None:
        return train, val, test
    elif val is not None:
        return train, val
    elif test is not None:
        return train, test
    else:
        return train

def time_series_train_test_split(X, y, test_size=10):
    """Split time series data maintaining temporal order"""
    split_idx = len(X) - test_size
    return (
        X.iloc[:split_idx], X.iloc[split_idx:],
        y.iloc[:split_idx], y.iloc[split_idx:])

def prepare_time_series_data(df_train, feature_set, target_col="Sales", horizon=10, winsorize=False, scaling=False):

    # Extract product data
    data = df_train.copy()
    
    y = data[[target_col]]  
    X = data.drop(columns=[target_col])
    
    # Select features - handle empty feature_set case
    if feature_set:
        try:
            # Convert feature_set to list if it's not already
            if not isinstance(feature_set, list):
                feature_set = [feature_set]
            # Select only columns that exist in X
            available_features = [f for f in feature_set if f in X.columns]
            X = X[available_features]
        except KeyError as e:
            raise ValueError(f"Some features not found in DataFrame: {e}")
    
    # Train/Test Split
    X_train, X_val, y_train, y_val = time_series_train_test_split(X, y, test_size=horizon)

    # Apply preprocessing steps
    y_train, y_val = preprocessing_pipeline(y_train, y_val, test=None, outlier_treatment=winsorize)

    # Scaling
    if scaling: 
        scaler_X = StandardScaler()
        X_train_scaled = scaler_X.fit_transform(X_train)
        X_val_scaled = scaler_X.transform(X_val)
        
        scaler_y = StandardScaler()
        y_train_scaled = scaler_y.fit_transform(y_train)
        y_val_scaled = scaler_y.transform(y_val)
        
        # Convert back to DataFrame for easier handling
        X_train_scaled = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns)
        X_val_scaled = pd.DataFrame(X_val_scaled, index=X_val.index, columns=X_val.columns)
        y_train_scaled = pd.DataFrame(y_train_scaled, index=y_train.index, columns=y_train.columns)
        y_val_scaled = pd.DataFrame(y_val_scaled, index=y_val.index, columns=y_val.columns)
        
        return X_train_scaled, X_val_scaled, y_train_scaled, y_val_scaled
    else:
        return X_train, X_val, y_train, y_val

In [30]:
def generate_feature_combinations(features, max_features=None):
    """Generate all possible feature combinations"""
    if max_features is None:
        max_features = len(features)
    
    all_combinations = []
    for r in range(1, max_features + 1):
        all_combinations.extend(combinations(features, r))
    
    return [list(comb) for comb in all_combinations]

In [31]:
def convert_results_to_df(results):
    data = []
    
    # Iterate over each result in the list
    for result in results:
        product_id = result.get('product_id', None)  # Safely get 'product_id'
        winsorize = result.get('winsorize', None)  # Safely get 'winsorize'
        
        # Handle empty 'features' list
        features = ', '.join(result['features']) if result['features'] else 'all'
        
        metrics = result.get('metrics', {})
        
        # Handle 'validation_predictions' list, use an empty list if missing
        validation_predictions = result.get('validation_predictions', [])
        
        # Iterate over each validation prediction and create a row in the DataFrame
        for pred in validation_predictions:
            data.append({
                'product_id': product_id,
                'winsorize': winsorize,
                'features': features,
                'RMSE': metrics.get('RMSE', np.nan),  # Use np.nan if RMSE is missing
                'MAPE': metrics.get('MAPE', np.nan),  # Use np.nan if MAPE is missing
                'R2': metrics.get('R2', np.nan),  # Use np.nan if R2 is missing
                'Overfit_Score': metrics.get('Overfit Score', np.nan),  # Use np.nan if 'Overfit Score' is missing
                'validation_prediction': pred
            })
    
    # Convert the list of dictionaries into a DataFrame
    df_results = pd.DataFrame(data)
    return df_results

### Roadmap:

✅ Check if Y is stationary

✅ Feature Selection → SHAP, RFE, or Correlation Matrix 

✅ Create a Pipeline → feature-engineering & AutoML 

✅ Base Models → ARIMA, XGBoost, Prophet, LSTMs

✅ Models Evaluation → RMSE, MAE, and hyperparameter tuning 

✅ Deployment → MLOps 

### **4. Modeling Pipeline (Feature selection, Scaling, Model testing):**

#### 4.1. Train vs Validation Splitting Strategy

Our pipeline is going to iterate over:

- **Rolling Forecast Validation (Expanding Window)**: The model is trained on an expanding dataset, making predictions on a fixed-size validation set.
Example: Train on 2010-2018 → Validate on 2019, then Train on 2010-2019 → Validate on 2020.
- **Walk-Forward Validation:** One-step forecasting where the model predicts the next step, and the validation window moves forward one step at a time.

#### 4.2. Scalers:
- Min-Max Scaling (0-1)
- Standard Scaling (Z-score, Mean=0, Std=1) 

#### 4.2. Feature Combinations

- No macro features (baseline model).
- Single-feature models.
- Multi-feature models.

Grid Search for Key Features rather than brute-force all combinations.???

#### 4.4. Models

First, base models and then we will tune hyperparameters of the best ones, wuth the best featues and best scalers, with the appropraite split method

| **Model**         | **Stationary Required?** | **Uses Lags?** | **Handles Trends?** | **Uses Macro Data?** | **Uses TensorFlow/Keras?** | **How?** |
|------------------|--------------------|------------|----------------|----------------|------------------|----------------------------|
| **ARIMA (Adapted ARIMAX)** | ❌ No (if differenced) | ❌ No | ✅ Yes (Differencing) | ✅ Yes (Exogenous Variables) | ❌ No | Traditional statistical method, but can handle macro data with ARIMAX |
| **Prophet**     | ❌ No | ✅ Yes (optional) | ✅ Yes | ✅ Yes | ❌ No | Handles trends/seasonality, allows external regressors |
| **NeuralProphet** | ❌ No | ✅ Yes | ✅ Yes | ✅ Yes | ❌ No | Deep learning-inspired improvement of Prophet |
| **XGBoost**     | ❌ No (Handles Trends) | ✅ Yes | ✅ Yes | ✅ Yes | ❌ No | Gradient boosting for structured time-series forecasting |
| **LSTMs**       | ❌ No (Learns Sequences) | ✅ Yes | ✅ Yes | ✅ Yes | ✅ Yes | Sequential modeling using TensorFlow/Keras |
| **TFT (Temporal Fusion Transformer)** | ❌ No | ✅ Yes | ✅ Yes | ✅ Yes | ✅ Yes | Uses attention mechanisms in TensorFlow/Keras for multivariate forecasting |
| **N-BEATS**     | ❌ No | ✅ Yes | ✅ Yes | ❌ No | ✅ Yes | Neural network-based forecasting using TensorFlow/Keras |
 add TimeGPT,  Google's TimesFM., LSTMs, Prophet,  NeuralProphet, ARIMA (Adapted ARIMAX)

#### 4.5. Evaluation Metrics: 
- RMSE → Measures error magnitude.
- MAPE → Percentage-based error, useful for business impact.
- R² → Measures how well the model explains variance.
- Overfit Score →  

$$
\text{Overfit Score} = \frac{\text{Test RMSE} - \text{Train RMSE}}{\text{Train RMSE}}
$$



    - < 0.1 (Underfit)
    - 0.1 - 0.5 (Good Fit)
    - > 0.5 (Overfit Warning!)

#### Modeling functions

In [33]:
def evaluate_model(model, X_train, y_train, X_test, y_test):
    # Predictions
    train_predictions = model.predict(X_train)
    test_predictions = model.predict(X_test)
    
    # Calculate metrics
    train_rmse = mean_squared_error(y_train, train_predictions, squared=False)
    test_rmse = mean_squared_error(y_test, test_predictions, squared=False)
    train_r2 = r2_score(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    # Print metrics
    print("=== Training Set ===")
    print(f"RMSE: {train_rmse:.3f}")
    print(f"MAPE: {train_mape*100:.2f}%")
    print(f"R²: {train_r2:.3f}\n")
    
    print("=== Test Set ===")
    print(f"RMSE: {test_rmse:.3f}")
    print(f"MAPE: {test_mape*100:.2f}%")
    print(f"R²: {test_r2:.3f}")
    
    return {
        'train': {'RMSE': train_rmse, 'MAPE': train_mape, 'R2': train_r2},
        'test': {'RMSE': test_rmse, 'MAPE': test_mape, 'R2': test_r2}}

In [35]:
def plot_model_performance(model, X_train, y_train, X_test, y_test, 
                          title="Model Performance", show_confidence=True):
    # Get predictions
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    
    # Create figure with subplots
    fig = make_subplots(rows=2, cols=1, 
                        subplot_titles=("Training Set Performance", "Test Set Performance"),
                        vertical_spacing=0.15)
    
    # Training set plot
    fig.add_trace(go.Scatter(
        x=X_train.index,
        y=y_train,
        mode='lines',
        name='Actual (Train)',
        line=dict(color='black', width=2)
    ), row=1, col=1)
    
    fig.add_trace(go.Scatter(
        x=X_train.index,
        y=train_pred,
        mode='lines',
        name='Predicted (Train)',
        line=dict(color='blue', width=1, dash='dash')
    ), row=1, col=1)
    
    # Test set plot
    fig.add_trace(go.Scatter(
        x=X_test.index,
        y=y_test,
        mode='lines',
        name='Actual (Test)',
        line=dict(color='black', width=2),
        showlegend=False
    ), row=2, col=1)
    
    fig.add_trace(go.Scatter(
        x=X_test.index,
        y=test_pred,
        mode='lines',
        name='Predicted (Test)',
        line=dict(color='red', width=1, dash='dash'),
        showlegend=False
    ), row=2, col=1)
    
    # Add confidence intervals if available and requested
    if show_confidence and hasattr(model, 'predict_interval'):
        try:
            _, train_lower, train_upper = model.predict_interval(X_train)
            fig.add_trace(go.Scatter(
                x=list(X_train.index) + list(X_train.index[::-1]),
                y=list(train_upper) + list(train_lower[::-1]),
                fill='toself',
                fillcolor='rgba(0,100,80,0.2)',
                line=dict(color='rgba(255,255,255,0)'),
                name='95% Confidence',
                showlegend=False
            ), row=1, col=1)
            
            _, test_lower, test_upper = model.predict_interval(X_test)
            fig.add_trace(go.Scatter(
                x=list(X_test.index) + list(X_test.index[::-1]),
                y=list(test_upper) + list(test_lower[::-1]),
                fill='toself',
                fillcolor='rgba(100,0,80,0.2)',
                line=dict(color='rgba(255,255,255,0)'),
                name='95% Confidence',
                showlegend=False
            ), row=2, col=1)
        except:
            pass
    
    # Update layout
    fig.update_layout(
        title=title,
        height=800,
        template="plotly_white",
        hovermode="x unified",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Value", row=1, col=1)
    fig.update_yaxes(title_text="Value", row=2, col=1)
    
    fig.show()

In [37]:
def plot_training_progress(history):
    """For models that provide training history (like neural networks)"""
    if not hasattr(history, 'history'):
        print("Model doesn't have training history to plot")
        return
    
    fig = go.Figure()
    
    # Plot training loss
    fig.add_trace(go.Scatter(
        x=np.arange(len(history.history['loss'])),
        y=history.history['loss'],
        mode='lines',
        name='Training Loss',
        line=dict(color='blue')
    ))
    
    # Plot validation loss if available
    if 'val_loss' in history.history:
        fig.add_trace(go.Scatter(
            x=np.arange(len(history.history['val_loss'])),
            y=history.history['val_loss'],
            mode='lines',
            name='Validation Loss',
            line=dict(color='orange')
        ))
    
    fig.update_layout(
        title="Training Progress",
        xaxis_title="Epoch",
        yaxis_title="Loss",
        template="plotly_white"
    )
    
    fig.show()

In [38]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import pandas as pd
from sklearn.metrics import (
    mean_squared_error, 
    r2_score, 
    mean_absolute_percentage_error,
    mean_absolute_error
)

def calculate_metrics(y_true, y_pred):
    """Calculate all relevant metrics for time series forecasting"""
    metrics = {
        'RMSE': mean_squared_error(y_true, y_pred, squared=False),
        'MAE': mean_absolute_error(y_true, y_pred),
        'MAPE': mean_absolute_percentage_error(y_true, y_pred) * 100,  # as percentage
        'R2': r2_score(y_true, y_pred),
        'Error_Std': np.std(y_true - y_pred)
    }
    return metrics

def print_metrics(metrics, title="Model Performance"):
    """Print metrics in a consistent format"""
    print(f"\n=== {title} ===")
    print(f"RMSE: {metrics['RMSE']:.3f}")
    print(f"MAE: {metrics['MAE']:.3f}")
    print(f"MAPE: {metrics['MAPE']:.2f}%")
    print(f"R²: {metrics['R2']:.3f}")
    print(f"Error Std: {metrics['Error_Std']:.3f}")

def plot_time_series_forecast(
    train_series, 
    test_series, 
    predictions,
    train_predictions=None,
    confidence_intervals=None,
    title="Forecast Performance",
    xaxis_title="Date",
    yaxis_title="Value"
):
    """
    Create interactive forecast visualization with Plotly
    
    Parameters:
    -----------
    train_series : pd.Series with training data
    test_series : pd.Series with actual test values
    predictions : array-like with predicted values
    train_predictions : array-like with predictions on training set (optional)
    confidence_intervals : tuple of (upper_bound, lower_bound) arrays
    """
    fig = go.Figure()
    
    # Training data
    fig.add_trace(go.Scatter(
        x=train_series.index,
        y=train_series,
        mode='lines',
        name='Training Data',
        line=dict(color='#1f77b4', width=2)
    ))
    
    # Training predictions if provided
    if train_predictions is not None:
        fig.add_trace(go.Scatter(
            x=train_series.index,
            y=train_predictions,
            mode='lines',
            name='Training Predictions',
            line=dict(color='#ff7f0e', width=1.5, dash='dot')
        ))
    
    # Test data
    fig.add_trace(go.Scatter(
        x=test_series.index,
        y=test_series,
        mode='lines',
        name='Actual Values',
        line=dict(color='#2ca02c', width=2)
    ))
    
    # Predictions
    fig.add_trace(go.Scatter(
        x=test_series.index,
        y=predictions,
        mode='lines',
        name='Predictions',
        line=dict(color='#d62728', width=2, dash='dash')
    ))
    
    # Confidence intervals if provided
    if confidence_intervals is not None:
        upper, lower = confidence_intervals
        fig.add_trace(go.Scatter(
            x=list(test_series.index) + list(test_series.index[::-1]),
            y=list(upper) + list(lower[::-1]),
            fill='toself',
            fillcolor='rgba(173, 216, 230, 0.2)',
            line=dict(color='rgba(255,255,255,0)'),
            name='95% Confidence',
            showlegend=True
        ))
    
    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        template="plotly_white",
        hovermode="x unified",
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        margin=dict(l=20, r=20, t=40, b=20)
    )
    
    return fig

def plot_training_history(
    history,
    metrics=['loss'],
    val_metrics=None,
    title="Training Progress"
):
    """
    Plot training history for models that provide it (like neural networks)
    
    Parameters:
    -----------
    history : keras History object or dict with training metrics
    metrics : list of metrics to plot from training
    val_metrics : list of corresponding validation metrics
    """
    if not hasattr(history, 'history') and not isinstance(history, dict):
        print("No training history available to plot")
        return None
    
    history_dict = history.history if hasattr(history, 'history') else history
    
    fig = go.Figure()
    
    # Plot training metrics
    for metric in metrics:
        if metric in history_dict:
            fig.add_trace(go.Scatter(
                x=np.arange(1, len(history_dict[metric])+1),
                y=history_dict[metric],
                mode='lines+markers',
                name=f'Training {metric}',
                line=dict(width=2)
            ))
    
    # Plot validation metrics if available
    if val_metrics:
        for val_metric in val_metrics:
            if val_metric in history_dict:
                fig.add_trace(go.Scatter(
                    x=np.arange(1, len(history_dict[val_metric])+1),
                    y=history_dict[val_metric],
                    mode='lines+markers',
                    name=f'Validation {val_metric}',
                    line=dict(width=2, dash='dash')
                ))
    
    fig.update_layout(
        title=title,
        xaxis_title="Epoch",
        yaxis_title="Metric Value",
        template="plotly_white",
        hovermode="x unified"
    )
    
    return fig

def evaluate_and_visualize(
    model,
    X_train, y_train,
    X_test, y_test,
    model_type='standard',
    confidence_intervals=None,
    training_history=None
):
    """
    Comprehensive evaluation and visualization wrapper
    
    Parameters:
    -----------
    model : fitted model
    X_train, y_train : training data
    X_test, y_test : test data
    model_type : str - 'standard', 'sequential', 'prophet', etc.
    confidence_intervals : tuple of (upper, lower) bounds
    training_history : history object for models that track training
    """
    # Get predictions based on model type
    if model_type == 'sequential':  # For Keras models
        train_pred = model.predict(X_train).flatten()
        test_pred = model.predict(X_test).flatten()
    elif model_type == 'prophet':
        train_pred = model.predict(X_train)['yhat'].values
        test_pred = model.predict(X_test)['yhat'].values
    else:  # Standard sklearn-style models
        train_pred = model.predict(X_train)
        test_pred = model.predict(X_test)
    
    # Calculate metrics
    train_metrics = calculate_metrics(y_train, train_pred)
    test_metrics = calculate_metrics(y_test, test_pred)
    
    # Print metrics
    print_metrics(train_metrics, "Training Set Performance")
    print_metrics(test_metrics, "Test Set Performance")
    
    # Create visualizations
    forecast_fig = plot_time_series_forecast(
        train_series=y_train,
        test_series=y_test,
        predictions=test_pred,
        train_predictions=train_pred,
        confidence_intervals=confidence_intervals,
        title="Model Forecast Performance"
    )
    
    if training_history is not None:
        history_fig = plot_training_history(training_history)
        return {
            'metrics': {'train': train_metrics, 'test': test_metrics},
            'forecast_plot': forecast_fig,
            'training_plot': history_fig
        }
    
    return {
        'metrics': {'train': train_metrics, 'test': test_metrics},
        'forecast_plot': forecast_fig
    }

#### 1. TimeGPT

In [17]:
nixtla_client = NixtlaClient(
    api_key = "nixak-CIwSKQ0cRLIuFR1TYllLFVakTGx3WCY30YPEKfxG0lDQcE0akGo3GE4aMJO9XXbkKjdFaGDP5x6uSxQ6"
)
nixtla_client.validate_api_key()

INFO:nixtla.nixtla_client:Happy Forecasting! :)


True

In [44]:
def calculate_metrics(y_true, y_pred, y_train=None, y_train_pred=None, print_metrics=False):
    metrics = {
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'MAPE': mean_absolute_percentage_error(y_true, y_pred) * 100,  # as percentage
        'R2': r2_score(y_true, y_pred)
    }
    
    if y_train is not None and y_train_pred is not None:
        train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
        metrics['Overfit Score'] = (metrics['RMSE'] - train_rmse) / max(train_rmse, 1e-10)
    
    if print_metrics:
        print("\n=== Metrics ===")
        print(f"RMSE: {metrics['RMSE']:.3f}")
        print(f"MAPE: {metrics['MAPE']:.2f}%")
        print(f"R²: {metrics['R2']:.3f}")
        if 'Overfit Score' in metrics:
            print(f"Overfit Score: {metrics['Overfit Score']:.3f}")
    
    return metrics

def convert_results_to_df(results):
    """Convert results to DataFrame format"""
    if results is None:
        return pd.DataFrame()
    
    return pd.DataFrame([{
        'product_id': results['product_id'],
        'winsorize': results['winsorize'],
        'features': ', '.join(results['features']) if results['features'] else 'all',
        'RMSE': results['metrics']['RMSE'],
        'MAPE': results['metrics']['MAPE'],
        'Overfit_Score': results['metrics'].get('Overfit Score', np.nan),
        'method': results['method']
    }])

In [72]:
def choose_parameters_timegpt(product_id, df_train, feature_set, target_col="Sales", 
                              nixtla_client=None, winsorize=False, 
                              horizon=7):
    
    # --- Input Validation ---
    if nixtla_client is None:
        raise ValueError("NixtlaClient instance is required.")
    if len(df_train) < 12:
        print(f"⚠️ Skipping {product_id} - only {len(df_train)} observations")
        return None
    
    # --- Data Preparation ---
    print('Preparing data...')
    try:
        X_train, X_val, y_train, y_val = prepare_time_series_data(df_train, feature_set, target_col, horizon, winsorize)

    except Exception as e:
        print(f"Data preparation failed: {str(e)}")
        return None
 
    # --- Determine Feature Types ---
    hist_exog = [f for f in feature_set if '_lag_' in f or '_ma_' in f] if feature_set else []
    fut_exog = [f for f in feature_set if f not in hist_exog] if feature_set else []


    # --- Prepare Future Exogenous Features ---
    X_future = None
    if fut_exog:
        X_future = X_val[fut_exog].copy()
        if not X_future.isnull().all().all():  
            X_future.insert(0, 'month_year', X_val.index)
        else:
            X_future = None  # Avoid passing empty DataFrame

    # --- Simple Forecast (12 ≤ obs < 36) ---
    if len(X_train) < 36:
        print(f"Using simple forecast ({len(X_train)} < 36 obs)")
        try:
            history = df_train.copy()
            if feature_set:
                history = history[feature_set + [target_col]]

            forecast = nixtla_client.forecast(df=history.reset_index(),
                                              time_col="month_year", target_col=target_col, h=horizon,
                                              X_df=X_future,  # Always pass X_future if available
                                              hist_exog_list=hist_exog if hist_exog else None)
            
            # Get training predictions
            train_pred = nixtla_client.forecast(df=history.reset_index(),time_col="month_year",
                                                target_col=target_col,h=1,hist_exog_list=hist_exog if hist_exog else None
                                                ).iloc[0]["TimeGPT"]
            
            train_predictions = [train_pred] * len(history)
            
            return {
                'product_id': product_id,'winsorize': winsorize,'features': feature_set,
                'metrics': calculate_metrics(y_true=y_val,y_pred=forecast["TimeGPT"],
                                             y_train=history[target_col],y_train_pred=train_predictions),
                'method': 'simple','validation_predictions': forecast["TimeGPT"]}
        except Exception as e:
            print(f"Simple forecast failed: {str(e)}")
            return None


    # --- Walk-Forward (≥36 obs) ---
    print(f"Using walk-forward ({len(X_train)} ≥ 36 obs)")
    predictions = []
    history = X_train.copy()
    history[target_col] = y_train.values
    history = history.reset_index()
    
    # Get training predictions
    try:
        train_fit = nixtla_client.forecast(df=history,time_col='month_year',target_col=target_col,h=1,
                                           hist_exog_list=hist_exog if hist_exog else None)
        train_predictions = [train_fit["TimeGPT"].iloc[0]] * len(y_train)
    except Exception as e:
        print(f"Training pred failed: {str(e)} - using last value")
        train_predictions = [y_train.iloc[-1]] * len(y_train)

    for i in range(horizon):
        try:
            X_future_step = None
            if fut_exog:
                X_future_step = X_val.iloc[[i]][fut_exog].copy()
                if not X_future_step.isnull().all().all():  
                    X_future_step.insert(0, 'month_year', X_val.index[i])
                else:
                    X_future_step = None  

            forecast = nixtla_client.forecast(df=history,time_col='month_year',target_col=target_col,
                                              h=1, X_df=X_future_step if fut_exog else None, # FUTURE features (macro)
                                              hist_exog_list=hist_exog if hist_exog else None)  # HISTORICAL features (lags, moving averages)
            
            pred = forecast["TimeGPT"].iloc[0]
            predictions.append(pred)
            
            new_row = X_val.iloc[i].copy()
            new_row['month_year'] = X_val.index[i]
            new_row[target_col] = pred
            history = pd.concat([history, pd.DataFrame([new_row])])
        
        except Exception as e:
            print(f"Iteration {i+1} failed: {str(e)}")
            raise
    
    metrics = calculate_metrics(y_true=y_val,y_pred=predictions,y_train=y_train,y_train_pred=train_predictions)
    
    return {'product_id': product_id,'winsorize': winsorize,'features': feature_set,'metrics': metrics,
            'method': 'walkforward','validation_predictions': predictions}


def find_best_timegpt_config(product_id, df_train, nixtla_client, lagged_df=None, target_col="Sales"):

    # Validate input data
    if not isinstance(df_train.index, pd.DatetimeIndex):
        raise TypeError("❌ df_train index is not a DatetimeIndex!")
    if df_train.index.isnull().any():
        raise ValueError("❌ Some index values could not be parsed into datetime!")
    
    results = []
    data = df_train.copy()
    
    # Feature groups for original data
    normal_features = [col for col in data.columns if col != target_col]
    normal_feature_combinations = generate_feature_combinations(normal_features, max_features=6)  
    normal_feature_combinations = [[]] + normal_feature_combinations
    print(normal_feature_combinations)
    
    # Test original data configurations
    for winsorize in [True, False]:
        for features in normal_feature_combinations:
            try:
                print(f"\nTesting - Winsorize: {winsorize}, Features: {features}")

                result = choose_parameters_timegpt(
                    product_id=product_id,df_train=data, feature_set=features, target_col=target_col,
                    nixtla_client=nixtla_client,winsorize=winsorize)
                
                if result:
                    print(f"✅ Success - RMSE: {result['metrics']['RMSE']:.2f}")
                    results.append(result)
            except Exception as e:
                print(f"⚠️ Configuration failed: {str(e)}")
                continue
    
    # Test lagged data configurations if available
    if lagged_df is not None:
        lagged_data = lagged_df.copy()
        lag_features = [col for col in lagged_data.columns if 'lag' in col]
        ma_features = [col for col in lagged_data.columns if '_ma_' in col]
        macro_features = [col for col in data.columns if col != target_col]
        
        # Generate lag feature combinations
        lag_feature_combinations = generate_feature_combinations(lag_features, max_features=5)
        
        # Create combined feature sets
        full_combinations = []
        for lag_combo in lag_feature_combinations:
            full_combinations.append(lag_combo)  # Just lags
            full_combinations.append(lag_combo + macro_features)  # Lags + macros
            full_combinations.append(lag_combo + ma_features)  # Lags + MAs
            full_combinations.append(lag_combo + macro_features + ma_features)  # All
        
        # Remove duplicates
        seen = set()
        unique_combinations = []
        for combo in full_combinations:
            combo_tuple = tuple(sorted(combo))
            if combo_tuple not in seen:
                seen.add(combo_tuple)
                unique_combinations.append(combo)
        
        # Test all unique combinations
        for winsorize in [True, False]:
            for features in unique_combinations:
                try:
                    print(f"\nTesting - Lagged Features: {features}")

                    result = choose_parameters_timegpt(product_id=product_id,df_train=lagged_data,feature_set=features,
                                                       target_col=target_col,nixtla_client=nixtla_client,winsorize=winsorize)
                    
                    if result:
                        print(f"✅ Success - RMSE: {result['metrics']['RMSE']:.2f}")
                        results.append(result)
                except Exception as e:
                    print(f"⚠️ Lagged configuration failed: {str(e)}")
                    continue
    
    if not results:
        raise ValueError(f"No valid configurations for {product_id}")
    
    # Select best configuration by RMSE
    best_config = min(results, key=lambda x: x['metrics']['RMSE'])
    
    print(f"\n✅ Best configuration for {product_id}:")
    print(f"- Winsorize: {best_config['winsorize']}")
    print(f"- Features: {best_config['features']}")
    print(f"- RMSE: {best_config['metrics']['RMSE']:.2f}")
    print(f"- Method: {best_config.get('method', 'walkforward')}")
    
    return best_config, results

def process_product_parallel(product_id):
    try:
        df_train = product_dfs[product_id].rename(columns={product_id: "Sales"})
        
        # Ensure DatetimeIndex
        if not isinstance(df_train.index, pd.DatetimeIndex):
            df_train.index = pd.to_datetime(df_train.index, errors="coerce")
        
        if df_train.index.isnull().any():
            raise ValueError("❌ Some index values could not be parsed into datetime!")

        lagged_df = lagged_product_dfs.get(product_id, None)

        best_config, results = find_best_timegpt_config(product_id=product_id,df_train=df_train,lagged_df=lagged_df,
            nixtla_client=nixtla_client,target_col="Sales")
        
        return best_config, results
    except Exception as e:
        print(f"⚠️ Failed to process {product_id}: {str(e)}")
        return None

In [22]:
all_product_ids = set(product_dfs.keys())

all_results = []  # List to store all results
best_configs = []  # List to store best configurations

for product_id in all_product_ids:
    print(f"Processing product: {product_id}")
    
    try:
        best_config_timegpt, results_timegpt = process_product_parallel(product_id)
        
        if best_config_timegpt is not None:
            best_configs.append(best_config_timegpt)  # Store best config
        else:
            print(f"Warning: No best config returned for product {product_id}")
        
        if results_timegpt is not None:
            all_results.extend(results_timegpt)  # Store results
        else:
            print(f"Warning: No results returned for product {product_id}")
            
    except Exception as e:
        print(f"Error processing product {product_id}: {str(e)}")

df_all_results = convert_results_to_df(all_results) if all_results else pd.DataFrame()
df_best_configs = pd.DataFrame(best_configs) if best_configs else pd.DataFrame()

df_all_results.to_csv("timegpt_best_configs.csv", index=False)
df_best_configs.to_csv("timegpt_best_configs_details.csv", index=False)

print("Processing completed! Results saved.")

INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: MS
WARNING - (py.warnings._showwarnmsg) - c:\Users\beatr\anaconda3\Lib\site-packages\nixtla\nixtla_client.py:383: UserWarning: `df` contains the following exogenous features: ['PRO27826_org', 'MAB_ELE_SHP392', 'MAB_ELE_SHP840', 'MAB_ELE_SHP276'], but `X_df` was not provided and they were not declared in `hist_exog_list`. They will be ignored.
  warnings.warn(

  warnings.warn(

INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Querying model metadata...


Processing product: P11
[[], ['PRO27826_org'], ['MAB_ELE_SHP392'], ['MAB_ELE_SHP840'], ['MAB_ELE_SHP276'], ['PRO27826_org', 'MAB_ELE_SHP392'], ['PRO27826_org', 'MAB_ELE_SHP840'], ['PRO27826_org', 'MAB_ELE_SHP276'], ['MAB_ELE_SHP392', 'MAB_ELE_SHP840'], ['MAB_ELE_SHP392', 'MAB_ELE_SHP276'], ['MAB_ELE_SHP840', 'MAB_ELE_SHP276'], ['PRO27826_org', 'MAB_ELE_SHP392', 'MAB_ELE_SHP840'], ['PRO27826_org', 'MAB_ELE_SHP392', 'MAB_ELE_SHP276'], ['PRO27826_org', 'MAB_ELE_SHP840', 'MAB_ELE_SHP276'], ['MAB_ELE_SHP392', 'MAB_ELE_SHP840', 'MAB_ELE_SHP276'], ['PRO27826_org', 'MAB_ELE_SHP392', 'MAB_ELE_SHP840', 'MAB_ELE_SHP276']]

Testing - Winsorize: True, Features: []
Preparing data...
Sales: Winsorized Bounds -> Lower = 88743.46, Upper = 3918670.23
Using walk-forward (36 ≥ 36 obs)


INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: MS
WARNING - (py.warnings._showwarnmsg) - c:\Users\beatr\anaconda3\Lib\site-packages\nixtla\nixtla_client.py:383: UserWarning: `df` contains the following exogenous features: ['PRO27826_org', 'MAB_ELE_SHP392', 'MAB_ELE_SHP840', 'MAB_ELE_SHP276'], but `X_df` was not provided and they were not declared in `hist_exog_list`. They will be ignored.
  warnings.warn(

  warnings.warn(

INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: MS
WARNING - (py.warnings._showwarnmsg) - c:\Users\beatr\anaconda3\Lib\site-packages\nixtla\nixtla_client.py:383: UserWarning: `df` contains the following exogenous features: ['PRO27826

✅ Success - RMSE: 1121699.56

Testing - Winsorize: True, Features: ['PRO27826_org']
Preparing data...
Sales: Winsorized Bounds -> Lower = 88743.46, Upper = 3918670.23
Using walk-forward (36 ≥ 36 obs)


INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: MS
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Using future exogenous features: ['PRO27826_org']
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: MS
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Using future exogenous features: ['PRO27826_org']
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: MS
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Using future exogenous features: ['PRO27826_org']
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: MS
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:U

...

INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: MS
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Using historical exogenous features: ['P12_lag_1', 'P12_lag_12', 'P12_lag_18', 'P12_ma_1', 'P12_ma_12', 'P12_ma_18']
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: MS
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Using future exogenous features: ['PRI27840_org', 'RohCOPPER1000_org', 'MAB_ELE_PRO156']
INFO:nixtla.nixtla_client:Using historical exogenous features: ['P12_lag_1', 'P12_lag_12', 'P12_lag_18', 'P12_ma_1', 'P12_ma_12', 'P12_ma_18']
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


✅ Success - RMSE: 111201.57

Testing - Lagged Features: ['P12_lag_1', 'P12_lag_12', 'P12_lag_18', 'PRI27840_org', 'RohCOPPER1000_org', 'MAB_ELE_PRO156', 'P12_ma_1', 'P12_ma_12', 'P12_ma_18']
Preparing data...
Using simple forecast (18 < 36 obs)


INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: MS
WARNING - (py.warnings._showwarnmsg) - c:\Users\beatr\anaconda3\Lib\site-packages\nixtla\nixtla_client.py:383: UserWarning: `df` contains the following exogenous features: ['PRI27840_org', 'RohCOPPER1000_org', 'MAB_ELE_PRO156'], but `X_df` was not provided and they were not declared in `hist_exog_list`. They will be ignored.
  warnings.warn(

  warnings.warn(

INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Using historical exogenous features: ['P12_lag_1', 'P12_lag_12', 'P12_lag_18', 'P12_ma_1', 'P12_ma_12', 'P12_ma_18']
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


✅ Success - RMSE: 111201.57

✅ Best configuration for P12:
- Winsorize: True
- Features: ['P12_lag_1']
- RMSE: 110218.04
- Method: simple


TypeError: list indices must be integers or slices, not str

In [23]:
print(best_configs)

[{'product_id': 'P11', 'winsorize': True, 'features': ['PRO27826_org', 'MAB_ELE_SHP392', 'MAB_ELE_SHP840'], 'metrics': {'RMSE': 1059588.239383447, 'MAPE': 190.62164938596192, 'R2': -0.5986571428127596, 'Overfit Score': -0.045103452391823645}, 'method': 'walkforward', 'validation_predictions': [2022097.4, 1707425.5, 1889917.0, 1667565.0, 2688938.5, 2508630.0, 1975512.6]}, {'product_id': 'P16', 'winsorize': True, 'features': ['P16_lag_3'], 'metrics': {'RMSE': 77236.74725928923, 'MAPE': 69.43729923357905, 'R2': -0.013261220876919166, 'Overfit Score': -0.54042951613273}, 'method': 'simple', 'validation_predictions': 0   176279.8300
1   176150.0600
2   173479.8900
3   171806.2800
4   169276.0300
5   168174.2500
6   166476.8400
Name: TimeGPT, dtype: float64}, {'product_id': 'P3', 'winsorize': True, 'features': ['PRO27826_org'], 'metrics': {'RMSE': 1400586.762485805, 'MAPE': 7.381212032997666, 'R2': -0.024034472082586023, 'Overfit Score': -0.5272839983616403}, 'method': 'walkforward', 'valida

In [53]:
def convert_results_to_df(results):
    if not results:
        return pd.DataFrame()  # Retorna um DataFrame vazio se results estiver vazio

    data = []

    # Itera sobre cada dicionário dentro da lista de resultados
    for result in results:
        product_id = result.get('product_id', None)
        winsorize = result.get('winsorize', None)
        
        # Verifica se 'features' existe e se não está vazio
        features = ', '.join(result.get('features', [])) if result.get('features') else 'all'
        
        metrics = result.get('metrics', {})

        validation_predictions = result.get('validation_predictions', [])

        # Adiciona uma linha para cada previsão de validação
        for pred in validation_predictions:
            data.append({
                'product_id': product_id,
                'winsorize': winsorize,
                'features': features,
                'RMSE': metrics.get('RMSE', np.nan),
                'MAPE': metrics.get('MAPE', np.nan),
                'Overfit_Score': metrics.get('Overfit Score', np.nan),
                'validation_prediction': pred
            })
    
    return pd.DataFrame(data)

In [30]:
df_timegpt = convert_results_to_df(best_configs)

In [31]:
df_timegpt.to_csv('best_configs_timegpt.csv', index=False)

#### 2. Google's TimesFM

In [ ]:
pip install transformers

   ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.2 MB 5.2 MB/s eta 0:00:02
   ---- ----------------------------------- 1.1/10.2 MB 11.1 MB/s eta 0:00:01
   ------ --------------------------------- 1.7/10.2 MB 12.0 MB/s eta 0:00:01
   -------- ------------------------------- 2.2/10.2 MB 11.5 MB/s eta 0:00:01
   ---------- ----------------------------- 2.6/10.2 MB 11.2 MB/s eta 0:00:01
   ------------- -------------------------- 3.5/10.2 MB 12.3 MB/s eta 0:00:01
   ---------------- ----------------------- 4.2/10.2 MB 12.7 MB/s eta 0:00:01
   ------------------ --------------------- 4.8/10.2 MB 13.2 MB/s eta 0:00:01
   -------------------- ------------------- 5.2/10.2 MB 12.2 MB/s eta 0:00:01
   ----------------------- ---------------- 6.1/10.2 MB 12.9 MB/s eta 0:00:01
   ------------------------- -------------- 6.5/10.2 MB 12.5 MB/s eta 0:00:01
   -------------------------- ------------- 6.8/10.2 MB 12.0 MB/s eta 0:0

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch

# Load TimesFM model
model = AutoModel.from_pretrained("google/timesfm")

# Load tokenizer (if applicable)
tokenizer = AutoTokenizer.from_pretrained("google/timesfm")

OSError: google/timesfm is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
def choose_parameters_timesfm(product_id, df_train, target_col='Sales', 
                              winsorize=False, feature_set=None, horizon=10):
    """
    Function to prepare data, run TimesFM forecast, and evaluate performance.
    """
    # Prepare data
    data = df_train[product_id].copy()
    y = data[target_col]
    X = data.drop(columns=[target_col])

    # Select features if specified
    if feature_set is not None:
        X = X[feature_set]

    # Time-based train/test split
    X_train, X_val, y_train, y_val = time_series_train_test_split(X, y, test_size=horizon)

    # Preprocessing
    train, val = preprocessing_pipeline(X_train, X_val, test=None, outlier_treatment=winsorize)

    # Scaling
    def scale(data_to_fit, data, method="standard"):
        return scaler_function(
            method,
            fit_data=data_to_fit, 
            df=data,
            numerical_columns=data_to_fit.columns)

    train = scale(train, train)
    val = scale(train, val)

    # Memory reduction
    train = reduce_memory_usage(train, train.columns)
    val = reduce_memory_usage(val, val.columns)

    # Select features
    train = train[feature_set]
    val = val[feature_set]

    # ---- TimesFM Processing ----
    # Convert to tensor format (assuming TimesFM expects PyTorch tensors)
    timeseries_tensor = torch.tensor(train.values, dtype=torch.float32).unsqueeze(0)

    # Run prediction using TimesFM
    with torch.no_grad():
        forecast_output = model(timeseries_tensor)

    # Extract predictions
    y_pred = forecast_output.squeeze(0).numpy()

    # Evaluate model
    metrics = calculate_metrics(y_val, y_pred)

    return {
        'product_id': product_id,
        'winsorize': winsorize,
        'features': feature_set if feature_set else 'all',
        'metrics': metrics,
        'predictions': y_pred}

def find_best_timesfm_config(product_id, df_train, target_col='Sales'):
    """
    Find the best TimesFM configuration by testing:
    - With/without winsorization
    - Different feature combinations
    """
    data = df_train[product_id].copy()
    available_features = [col for col in data.columns if col != target_col]

    # Generate all possible feature combinations (max 3 features at a time)
    feature_combinations = generate_feature_combinations(available_features, max_features=3)

    # Test all configurations
    results = []
    for winsorize in [True, False]:
        for features in feature_combinations:
            print(f"\nTesting config - Winsorize: {winsorize}, Features: {features}")

            result = choose_parameters_timesfm(
                product_id=product_id,
                df_train=df_train,
                target_col=target_col,
                winsorize=winsorize,
                feature_set=features
            )
            results.append(result)

    # Find best configuration (lowest RMSE)
    results_df = pd.DataFrame(results)
    best_idx = results_df['metrics'].apply(lambda x: x['RMSE']).idxmin()
    best_config = results[best_idx]

    print(f"\nBest configuration for product {product_id}:")
    print(f"- Winsorize: {best_config['winsorize']}")
    print(f"- Features: {best_config['features']}")
    print(f"- RMSE: {best_config['metrics']['RMSE']:.2f}")
    print(f"- R²: {best_config['metrics']['R2']:.4f}")

    return best_config

In [74]:
# List to store results
comparison_results = []

# List of all product IDs (assuming df_train has product_id as a column)
all_product_ids = df_train.columns.levels[0]  # Adjust this if product_id is structured differently

# Apply TimesFM model for each product
for product_id in all_product_ids:
    print(f"\n🔄 Processing Product ID: {product_id} ...")

    best_config = find_best_timesfm_config(
        product_id=product_id,
        df_train=df_train,
        target_col='Sales')

    # Save results in the comparison table
    comparison_results.append({
        'Product_ID': product_id,
        'Model': 'TimesFM',
        'Winsorize': best_config['winsorize'],
        'Features': best_config['features'],
        'RMSE': best_config['metrics']['RMSE'],
        'R²': best_config['metrics']['R2']})

# Convert results to DataFrame for analysis
comparison_df = pd.DataFrame(comparison_results)

# Save as CSV for further analysis
comparison_df.to_csv("timesfm_results.csv", index=False)

# Display results
print("\n📊 TimesFM Model Performance Across Products:")
print(comparison_df)

NameError: name 'df_train' is not defined

#### 3. XGboost

In [75]:
def choose_parameters_xgboost(product_id, df_train, feature_set, target_col="Sales", 
                            param_grid=None, n_jobs=-1, winsorize=False, horizon=7):
    """Optimized XGBoost forecasting with proper walk-forward validation"""
    
    # --- Input Validation ---
    if len(df_train) < 12:
        print(f"⚠️ Skipping {product_id} - only {len(df_train)} observations")
        return None
    
    # --- Data Preparation ---
    print('Preparing data...')
    try:
        X_train, X_val, y_train, y_val = prepare_time_series_data(
            df_train, feature_set, target_col, horizon, winsorize)
        # Verify temporal ordering
        assert X_train.index[-1] < X_val.index[0], "Validation data must be after training data"
    except Exception as e:
        print(f"Data preparation failed: {str(e)}")
        return None

    # --- Optimized Parameter Grid for Small Data ---
    if param_grid is None:
        param_grid = {
            'n_estimators': [50, 100],
            'max_depth': [2, 3],
            'learning_rate': [0.01, 0.05, 0.1],
            'subsample': [0.9, 1.0],
            'colsample_bytree': [0.9, 1.0],
            'min_child_weight': [1, 3]}

    # --- Efficient Model Setup ---
    base_model = xgb.XGBRegressor(
        objective='reg:squarederror',random_state=42,n_jobs=1,tree_method='hist',  # Faster training
        enable_categorical=False,gamma=0,reg_alpha=0,reg_lambda=1)

    # --- Strict Temporal Validation ---
    test_fold = np.array([-1] * len(X_train) + [0] * len(X_val))
    ps = PredefinedSplit(test_fold)

    # --- Focused Grid Search ---
    grid_search = GridSearchCV(
        estimator=base_model,param_grid=param_grid,scoring='neg_root_mean_squared_error',cv=ps,n_jobs=n_jobs,verbose=1,refit=True)

    # --- Training with Validation ---
    X_combined = pd.concat([X_train, X_val])
    y_combined = pd.concat([y_train, y_val])
    
    print(f"Starting grid search for {product_id}...")
    try:
        grid_search.fit(X_combined, y_combined)
        best_model = grid_search.best_estimator_
    except Exception as e:
        print(f"Grid search failed: {str(e)}")
        return None

    # --- Robust Walk-forward Validation ---
    predictions = []
    print(f"Using walk-forward (n_train={len(X_train)}, horizon={horizon})")
    
    for i in range(horizon):
        try:
            # Create expanding window
            window_X = pd.concat([X_train, X_val.iloc[:i]])
            window_y = pd.concat([y_train, y_val.iloc[:i]])
            
            # Correct XGBoost 2.1.3+ fitting
            best_model.fit(window_X, window_y, eval_set=[(X_val.iloc[[i]], y_val.iloc[[i]])], verbose=0)
    
            pred = best_model.predict(X_val.iloc[[i]]).item()
            predictions.append(pred)
            
        except Exception as e:
            print(f"Iteration {i+1} failed: {str(e)} - using fallback")
            fallback = (predictions[-1] if len(predictions) > 0 else 
                       y_train.iloc[-1] if len(y_train) > 0 else np.nan)
            predictions.append(fallback)

    # --- Training Metrics ---
    try:
        train_predictions = best_model.predict(X_train).tolist()
    except Exception as e:
        print(f"Training predictions failed: {str(e)} - using last value")
        train_predictions = [y_train.iloc[-1]] * len(y_train)

    # --- Comprehensive Metrics ---
    metrics = calculate_metrics(y_true=y_val,y_pred=predictions,y_train=y_train,y_train_pred=train_predictions)
    
    return {
        'product_id': product_id,
        'winsorize': winsorize,
        'features': feature_set,
        'metrics': metrics,
        'method': 'walkforward',
        'best_params': grid_search.best_params_,
        'validation_predictions': predictions,
        'feature_importances': dict(zip(feature_set, best_model.feature_importances_))}


def find_best_xgboost_config(product_id, df_train, lagged_df=None, target_col="Sales", 
                            n_jobs=-1, custom_param_grids=None):
    # Validate input data
    if not isinstance(df_train.index, pd.DatetimeIndex):
        raise TypeError("❌ df_train index is not a DatetimeIndex!")
    if df_train.index.isnull().any():
        raise ValueError("❌ Some index values could not be parsed into datetime!")
    
    results = []
    data = df_train.copy()
    
    # Feature groups for original data
    normal_features = [col for col in data.columns if col != target_col]
    normal_feature_combinations = generate_feature_combinations(normal_features, max_features=6)  
    normal_feature_combinations = [[]] + normal_feature_combinations
    
    # Test original data configurations
    for winsorize in [True, False]:
        for features in normal_feature_combinations:
            try:
                print(f"\nTesting - Winsorize: {winsorize}, Features: {features}")

                # Use custom param grid if provided for this feature set
                param_grid = None
                if custom_param_grids and tuple(features) in custom_param_grids:
                    param_grid = custom_param_grids[tuple(features)]

                result = choose_parameters_xgboost(product_id=product_id,df_train=data,feature_set=features,
                    target_col=target_col,winsorize=winsorize,n_jobs=n_jobs,param_grid=param_grid)
                
                if result:
                    print(f"✅ Success - RMSE: {result['metrics']['RMSE']:.2f}")
                    results.append(result)
            except Exception as e:
                print(f"⚠️ Configuration failed: {str(e)}")
                continue
    
    # Test lagged data configurations if available
    if lagged_df is not None:
        lagged_data = lagged_df.copy()
        lag_features = [col for col in lagged_data.columns if 'lag' in col]
        ma_features = [col for col in lagged_data.columns if '_ma_' in col]
        macro_features = [col for col in data.columns if col != target_col]
        
        # Generate lag feature combinations
        lag_feature_combinations = generate_feature_combinations(lag_features, max_features=5)
        
        # Create combined feature sets
        full_combinations = []
        for lag_combo in lag_feature_combinations:
            full_combinations.append(lag_combo)  # Just lags
            full_combinations.append(lag_combo + macro_features)  # Lags + macros
            full_combinations.append(lag_combo + ma_features)  # Lags + MAs
            full_combinations.append(lag_combo + macro_features + ma_features)  # All
        
        # Remove duplicates
        seen = set()
        unique_combinations = []
        for combo in full_combinations:
            combo_tuple = tuple(sorted(combo))
            if combo_tuple not in seen:
                seen.add(combo_tuple)
                unique_combinations.append(combo)
        
        # Test all unique combinations
        for winsorize in [True, False]:
            for features in unique_combinations:
                try:
                    print(f"\nTesting - Lagged Features: {features}")

                    # Use custom param grid if provided for this feature set
                    param_grid = None
                    if custom_param_grids and tuple(features) in custom_param_grids:
                        param_grid = custom_param_grids[tuple(features)]

                    result = choose_parameters_xgboost(product_id=product_id,df_train=lagged_data,feature_set=features,
                                                       target_col=target_col,winsorize=winsorize,n_jobs=n_jobs,param_grid=param_grid)
                    
                    if result:
                        print(f"✅ Success - RMSE: {result['metrics']['RMSE']:.2f}")
                        results.append(result)
                except Exception as e:
                    print(f"⚠️ Lagged configuration failed: {str(e)}")
                    continue
    
    if not results:
        raise ValueError(f"No valid configurations for {product_id}")
    
    # Select best configuration by RMSE
    best_config = min(results, key=lambda x: x['metrics']['RMSE'])
    
    print(f"\n✅ Best configuration for {product_id}:")
    print(f"- Winsorize: {best_config['winsorize']}")
    print(f"- Features: {best_config['features']}")
    print(f"- Best params: {best_config['best_params']}")
    print(f"- RMSE: {best_config['metrics']['RMSE']:.2f}")
    print(f"- Method: {best_config.get('method', 'walkforward')}")
    
    return best_config, results

def process_product_parallel(product_id, custom_param_grids=None):
    try:
        df_train = product_dfs[product_id].rename(columns={product_id: "Sales"})
        
        # Ensure DatetimeIndex
        if not isinstance(df_train.index, pd.DatetimeIndex):
            df_train.index = pd.to_datetime(df_train.index, errors="coerce")
        
        if df_train.index.isnull().any():
            raise ValueError("❌ Some index values could not be parsed into datetime!")

        lagged_df = lagged_product_dfs.get(product_id, None)

        best_config, results = find_best_xgboost_config(product_id=product_id,df_train=df_train,lagged_df=lagged_df,target_col="Sales", custom_param_grids=custom_param_grids)
        
        return best_config, results
    except Exception as e:
        print(f"⚠️ Failed to process {product_id}: {str(e)}")
        return None

In [ ]:
custom_param_grids= ver 

In [117]:
all_product_ids ={'P14'}

In [ ]:
all_product_ids

In [37]:
all_product_ids = set(product_dfs.keys())
all_results_xgboost = []  # List to store all results
best_configs_xgboost = []  # List to store best configurations

for product_id in all_product_ids:
    print(f"Processing product: {product_id}")
    
    try:
        best_config_xgboost, results_xgboost = process_product_parallel(product_id, custom_param_grids=None)
        
        if best_config_xgboost is not None:
            best_configs_xgboost.append(best_config_xgboost)  # Store best config
        else:
            print(f"Warning: No best config returned for product {product_id}")
        
        if results_xgboost is not None:
            all_results_xgboost.extend(results_xgboost)  # Store results
        else:
            print(f"Warning: No results returned for product {product_id}")
            
    except Exception as e:
        print(f"Error processing product {product_id}: {str(e)}")

Processing product: P11

Testing - Winsorize: True, Features: []
Preparing data...
Sales: Winsorized Bounds -> Lower = 88743.46, Upper = 3918670.23
Starting grid search for P11...
Fitting 1 folds for each of 96 candidates, totalling 96 fits
Using walk-forward (n_train=36, horizon=7)
✅ Success - RMSE: 796282.59

Testing - Winsorize: True, Features: ['PRO27826_org']
Preparing data...
Sales: Winsorized Bounds -> Lower = 88743.46, Upper = 3918670.23
Starting grid search for P11...
Fitting 1 folds for each of 96 candidates, totalling 96 fits
Using walk-forward (n_train=36, horizon=7)
✅ Success - RMSE: 793425.89

Testing - Winsorize: True, Features: ['MAB_ELE_SHP392']
Preparing data...
Sales: Winsorized Bounds -> Lower = 88743.46, Upper = 3918670.23
Starting grid search for P11...
Fitting 1 folds for each of 96 candidates, totalling 96 fits
Using walk-forward (n_train=36, horizon=7)
✅ Success - RMSE: 744546.94

Testing - Winsorize: True, Features: ['MAB_ELE_SHP840']
Preparing data...
Sales: 

In [54]:
df_all_results_xgboost = convert_results_to_df(all_results_xgboost) if all_results_xgboost else pd.DataFrame()
df_best_configs_xgboost = pd.DataFrame(best_configs_xgboost) if best_configs_xgboost else pd.DataFrame()

df_all_results_xgboost.to_csv("xgboost_results.csv", index=False)
df_best_configs_xgboost.to_csv("xgboost_best_configs_details.csv", index=False)

print("Processing completed! Results saved.")

NameError: name 'all_results_xgboost' is not defined

#### 4. TCN

In [46]:
def choose_parameters_tcn(product_id, df_train, feature_set, target_col="Sales", 
                         param_grid=None, n_jobs=-1, winsorize=False, horizon=7):
    """Optimized TCN forecasting with walk-forward validation"""
    
    if len(df_train) < 12:
        print(f"⚠️ Skipping {product_id} - only {len(df_train)} observations")
        return None
    
    print('Preparing data...')
    try:
        X_train, X_val, y_train, y_val = prepare_time_series_data(
            df_train, feature_set, target_col, horizon, winsorize, scaling=True)
        
        assert X_train.index[-1] < X_val.index[0], "Validation data must be after training data"
    except Exception as e:
        print(f"Data preparation failed: {str(e)}")
        return None

    # Reshape data for TCN input (samples, timesteps, features)
    X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_val = X_val.values.reshape((X_val.shape[0], 1, X_val.shape[1]))
    y_train = y_train.values.reshape(-1, 1)
    y_val = y_val.values.reshape(-1, 1)

    if param_grid is None:
        param_grid = {
            'nb_filters': [16, 32],
            'kernel_size': [2, 3],
            'dilations': [[1, 2, 4], [1, 2, 4, 8]],
            'dropout_rate': [0.1, 0.2],
            'return_sequences': [False]  # single-step prediction
}
    
    def build_tcn(nb_filters=32, kernel_size=2, dilations=[1, 2, 4], 
                 dropout_rate=0.1, return_sequences=False):
        inputs = Input(shape=(1, X_train.shape[2]))
        x = TCN(nb_filters=nb_filters, kernel_size=kernel_size, 
               dilations=dilations, dropout_rate=dropout_rate,
               return_sequences=return_sequences)(inputs)
        outputs = Dense(1)(x)
        model = Model(inputs, outputs)
        model.compile(optimizer="adam", loss="mse")
        return model
    
    best_params = None
    best_rmse = float('inf')

    for filters in param_grid['nb_filters']:
        for kernel in param_grid['kernel_size']:
            for dilations in param_grid['dilations']:
                for dropout in param_grid['dropout_rate']:
                    for return_seq in param_grid['return_sequences']:
                        model = build_tcn(filters, kernel, dilations, dropout, return_seq)
                        try:
                            model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=0)
                            
                            # Walk-forward validation (predict one step at a time)
                            predictions = []
                            history_X = X_train.copy()
                            history_y = y_train.copy()
                            
                            for i in range(len(X_val)):
                                # Predict next step
                                X_next = X_val[i].reshape(1, 1, -1)
                                pred = model.predict(X_next, verbose=0).flatten()[0]
                                predictions.append(pred)
                                
                            # Calculate RMSE
                            if len(y_val) == len(predictions):
                                rmse = np.sqrt(np.mean((y_val.flatten() - predictions) ** 2))
                            else:
                                print(f"Shape mismatch: y_val ({y_val.shape}), preds ({len(predictions)})")
                                continue
                            
                            if rmse < best_rmse:
                                best_rmse = rmse
                                best_params = {
                                    'nb_filters': filters,
                                    'kernel_size': kernel,
                                    'dilations': dilations,
                                    'dropout_rate': dropout,
                                    'return_sequences': return_seq
                                }
                        except Exception as e:
                            print(f"Hyperparameter set failed: {str(e)}")

    if best_params is None:
        print(f"❌ No valid models for {product_id}")
        return None

    # Final model training and walk-forward prediction
    final_model = build_tcn(**best_params)
    final_model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=0)
    predictions = []

    # FIXED: Create DataFrame with both features and target
    history_data = np.column_stack([X_train[:, 0, :], y_train.flatten()])
    history = pd.DataFrame(history_data, columns=feature_set + [target_col])
    history.index = range(len(history))
    
    for i in range(horizon):
        try:
            X_future_step = X_val[i].reshape(1, 1, -1)
            pred = final_model.predict(X_future_step, verbose=0).flatten()[0]
            predictions.append(pred)
            
            # Update training data
            new_row = X_val[i, 0, :].tolist()
            new_row.append(pred)
            new_df = pd.DataFrame([new_row], columns=feature_set + [target_col])
            history = pd.concat([history, new_df]).reset_index(drop=True)
            
            # Retrain on expanded window
            rolling_window = 36 if len(history) > 36 else len(history)
            train_subset = history.iloc[-rolling_window:]
            
            X_train_update = train_subset[feature_set].values.reshape(-1, 1, len(feature_set))
            y_train_update = train_subset[target_col].values.reshape(-1, 1)
            
            final_model.fit(X_train_update, y_train_update, epochs=5, batch_size=16, verbose=0)
        
        except Exception as e:
            print(f"Iteration {i+1} failed: {str(e)}")
            predictions.append(predictions[-1] if predictions else y_train[-1][0])
    
    metrics = calculate_metrics(y_true=y_val.flatten(), y_pred=predictions, y_train=y_train.flatten())
    
    return {
        'product_id': product_id,
        'winsorize': winsorize,
        'features': feature_set,
        'metrics': metrics,
        'method': 'walkforward',
        'best_params': best_params,
        'validation_predictions': predictions
    }

def find_best_tcn_config(product_id, df_train, lagged_df=None, target_col="Sales", 
                            n_jobs=-1, custom_param_grids=None):
    # Validate input data
    if not isinstance(df_train.index, pd.DatetimeIndex):
        raise TypeError("❌ df_train index is not a DatetimeIndex!")
    if df_train.index.isnull().any():
        raise ValueError("❌ Some index values could not be parsed into datetime!")
    
    results = []
    data = df_train.copy()
    
    # Feature groups for original data
    normal_features = [col for col in data.columns if col != target_col]
    normal_feature_combinations = generate_feature_combinations(normal_features, max_features=6)  
    normal_feature_combinations = normal_feature_combinations
    
    # Test original data configurations
    for winsorize in [True, False]:
        for features in normal_feature_combinations:
            try:
                print(f"\nTesting - Winsorize: {winsorize}, Features: {features}")

                # Use custom param grid if provided for this feature set
                param_grid = None
                if custom_param_grids and tuple(features) in custom_param_grids:
                    param_grid = custom_param_grids[tuple(features)]

                result = choose_parameters_tcn(product_id=product_id,df_train=data,feature_set=features,
                    target_col=target_col,winsorize=winsorize,n_jobs=n_jobs,param_grid=param_grid)
                
                if result:
                    print(f"✅ Success - RMSE: {result['metrics']['RMSE']:.2f}")
                    results.append(result)
            except Exception as e:
                print(f"⚠️ Configuration failed: {str(e)}")
                continue
    
    # Test lagged data configurations if available
    if lagged_df is not None:
        lagged_data = lagged_df.copy()
        lag_features = [col for col in lagged_data.columns if 'lag' in col]
        ma_features = [col for col in lagged_data.columns if '_ma_' in col]
        macro_features = [col for col in data.columns if col != target_col]
        
        # Generate lag feature combinations
        lag_feature_combinations = generate_feature_combinations(lag_features, max_features=5)
        
        # Create combined feature sets
        full_combinations = []
        for lag_combo in lag_feature_combinations:
            full_combinations.append(lag_combo)  # Just lags
            full_combinations.append(lag_combo + macro_features)  # Lags + macros
            full_combinations.append(lag_combo + ma_features)  # Lags + MAs
            full_combinations.append(lag_combo + macro_features + ma_features)  # All
        
        # Remove duplicates
        seen = set()
        unique_combinations = []
        for combo in full_combinations:
            combo_tuple = tuple(sorted(combo))
            if combo_tuple not in seen:
                seen.add(combo_tuple)
                unique_combinations.append(combo)
        
        # Test all unique combinations
        for winsorize in [True, False]:
            for features in unique_combinations:
                try:
                    print(f"\nTesting - Lagged Features: {features}")

                    # Use custom param grid if provided for this feature set
                    param_grid = None
                    if custom_param_grids and tuple(features) in custom_param_grids:
                        param_grid = custom_param_grids[tuple(features)]

                    result = choose_parameters_tcn(product_id=product_id,df_train=lagged_data,feature_set=features,
                                                       target_col=target_col,winsorize=winsorize,n_jobs=n_jobs,param_grid=param_grid)
                    
                    if result:
                        print(f"✅ Success - RMSE: {result['metrics']['RMSE']:.2f}")
                except Exception as e:
                    print(f"⚠️ Lagged configuration failed: {str(e)}")
                    continue
    
    if not results:
        raise ValueError(f"No valid configurations for {product_id}")
    
    # Select best configuration by RMSE
    best_config = min(results, key=lambda x: x['metrics']['RMSE'])
    
    print(f"\n✅ Best configuration for {product_id}:")
    print(f"- Winsorize: {best_config['winsorize']}")
    print(f"- Features: {best_config['features']}")
    print(f"- Best params: {best_config['best_params']}")
    print(f"- RMSE: {best_config['metrics']['RMSE']:.2f}")
    print(f"- Method: {best_config.get('method', 'walkforward')}")
    
    return best_config, results

def process_product_parallel(product_id, custom_param_grids=None):
    try:
        df_train = product_dfs[product_id].rename(columns={product_id: "Sales"})
        
        # Ensure DatetimeIndex
        if not isinstance(df_train.index, pd.DatetimeIndex):
            df_train.index = pd.to_datetime(df_train.index, errors="coerce")
        
        if df_train.index.isnull().any():
            raise ValueError("❌ Some index values could not be parsed into datetime!")

        lagged_df = lagged_product_dfs.get(product_id, None)
        best_config, results = find_best_tcn_config(product_id=product_id,df_train=df_train,lagged_df=lagged_df,target_col="Sales", custom_param_grids=custom_param_grids)
        
        return best_config, results
    except Exception as e:
        print(f"⚠️ Failed to process {product_id}: {str(e)}")
        return None

In [47]:
all_results_tcn = []  # List to store all results
best_configs_tcn = []  # List to store best configurations

In [48]:
all_product_ids = set(list(product_dfs.keys()))
all_product_ids

{'P1',
 'P11',
 'P12',
 'P13',
 'P14',
 'P16',
 'P20',
 'P3',
 'P36',
 'P4',
 'P5',
 'P6',
 'P8',
 'P9',
 'Sales_CPI'}

In [49]:
all_product_ids = set(list(product_dfs.keys())[:1])

for product_id in all_product_ids:
    print(f"Processing product: {product_id}")
    
    try:
        best_config_tcn, results_tcn = process_product_parallel(product_id, custom_param_grids=None)
        
        if best_config_tcn is not None:
            best_configs_tcn.append(best_config_tcn)  # Store best config
        else:
            print(f"Warning: No best config returned for product {product_id}")
        
        if results_tcn is not None:
            all_results_tcn.extend(results_tcn)  # Store results
        else:
            print(f"Warning: No results returned for product {product_id}")
            
    except Exception as e:
        print(f"Error processing product {product_id}: {str(e)}")

Processing product: P1

Testing - Winsorize: True, Features: ['MAB_ELE_SHP840']
Preparing data...
Sales: Winsorized Bounds -> Lower = 12773011.92, Upper = 41955264.27
✅ Success - RMSE: 0.61

Testing - Winsorize: True, Features: ['PRI27276_org']
Preparing data...
Sales: Winsorized Bounds -> Lower = 12773011.92, Upper = 41955264.27
✅ Success - RMSE: 0.96

Testing - Winsorize: True, Features: ['PRO27826_org']
Preparing data...
Sales: Winsorized Bounds -> Lower = 12773011.92, Upper = 41955264.27
✅ Success - RMSE: 0.76

Testing - Winsorize: True, Features: ['MAB_ELE_PRO276']
Preparing data...
Sales: Winsorized Bounds -> Lower = 12773011.92, Upper = 41955264.27
✅ Success - RMSE: 0.79

Testing - Winsorize: True, Features: ['MAB_ELE_SHP1100']
Preparing data...
Sales: Winsorized Bounds -> Lower = 12773011.92, Upper = 41955264.27
✅ Success - RMSE: 0.82

Testing - Winsorize: True, Features: ['MAB_ELE_SHP840', 'PRI27276_org']
Preparing data...
Sales: Winsorized Bounds -> Lower = 12773011.92, Upper

In [50]:
print(all_results_tcn)
print(best_configs_tcn)

[{'product_id': 'P1', 'winsorize': True, 'features': ['MAB_ELE_SHP840'], 'metrics': {'RMSE': 0.6100804553383123, 'MAPE': 453.19752230944783, 'R2': -0.39713168659241505}, 'method': 'walkforward', 'best_params': {'nb_filters': 16, 'kernel_size': 2, 'dilations': [1, 2, 4], 'dropout_rate': 0.2, 'return_sequences': False}, 'validation_predictions': [0.3246293, 0.26861608, 0.41924566, 0.2183412, 0.3289858, 0.4532824, 0.4222339]}, {'product_id': 'P1', 'winsorize': True, 'features': ['PRI27276_org'], 'metrics': {'RMSE': 0.9607105028120426, 'MAPE': 974.8896646154628, 'R2': -2.4645608658583407}, 'method': 'walkforward', 'best_params': {'nb_filters': 16, 'kernel_size': 3, 'dilations': [1, 2, 4], 'dropout_rate': 0.1, 'return_sequences': False}, 'validation_predictions': [0.5148098, 0.5702051, 1.0821116, 1.590806, 1.9925718, 2.0422704, 2.1319296]}, {'product_id': 'P1', 'winsorize': True, 'features': ['PRO27826_org'], 'metrics': {'RMSE': 0.7599837669413102, 'MAPE': 328.4460563418336, 'R2': -1.168062

In [55]:
df_all_results_tcn = convert_results_to_df(all_results_tcn) if all_results_tcn else pd.DataFrame()
df_best_configs_tcn = pd.DataFrame(best_configs_tcn) if best_configs_tcn else pd.DataFrame()

df_all_results_tcn.to_csv("tcn_results.csv", index=False)
df_best_configs_tcn.to_csv("tcn_best_configs_details.csv", index=False)

print("Processing completed! Results saved.")

Processing completed! Results saved.


#### 5. Arima

In [20]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from pmdarima import auto_arima
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error

# Ensure time-series data is sorted and complete
df_p1_lags = df_p1_lags.sort_index()
df_p1_lags = df_p1_lags.asfreq('MS').interpolate()  # Fill missing months

# Fit Auto ARIMA Model (Using all data initially)
auto_model = auto_arima(
    df_p1_lags['P1'].dropna(),  
    exogenous=df_p1_lags[['P1_lag_1', 'P1_lag_2', 'P1_lag_3']].dropna(),  
    seasonal=False, 
    trace=True,
    stepwise=True,
    suppress_warnings=True,
    error_action='ignore',
    max_p=5, 
    max_q=5,
    d=None,  # Let ADF test decide differencing
    test='adf',  
    scoring='mse'
)
print(auto_model.summary())

# Walk-Forward Forecasting with Expanding Window
def walk_forward_forecast(model, df, horizon=10, initial_train_size=33):
    """
    Walk-forward validation using an expanding window approach.
    Predicts 'horizon' steps ahead at each iteration.
    """
    predictions, true_values, lower_bounds, upper_bounds = [], [], [], []

    for i in range(len(df) - initial_train_size - horizon + 1):
        train = df.iloc[: i + initial_train_size]  # Expanding training set
        test = df.iloc[i + initial_train_size : i + initial_train_size + horizon]

        model.fit(train['P1'])
        pred, conf_int = model.predict(n_periods=len(test), return_conf_int=True)

        if len(pred) == len(test):  # Ensure consistent shape
            predictions.append(pred)
            true_values.append(test['P1'].values)
            lower_bounds.append(conf_int[:, 0])
            upper_bounds.append(conf_int[:, 1])

    return np.array(true_values), np.array(predictions), np.array(lower_bounds), np.array(upper_bounds)

# Perform Walk-Forward Forecasting
true_values, predictions, lower_bounds, upper_bounds = walk_forward_forecast(auto_model, df_p1_lags, horizon=10)

# Compute Errors
def print_error(true, pred):
    RMSE = mean_squared_error(true.flatten(), pred.flatten(), squared=False)
    R2 = r2_score(true.flatten(), pred.flatten())
    MAPE = mean_absolute_percentage_error(true.flatten(), pred.flatten())
    print("RMSE:", RMSE)
    print("MAPE:", round(MAPE * 100, 2), "%")
    print("R2:", R2)

print_error(true_values, predictions)

# ✅ Plot Full Dataset + Predictions + Confidence Intervals
fig = go.Figure()

# Add the real data (Training + Test)
fig.add_trace(go.Scatter(
    x=df_p1_lags.index,
    y=df_p1_lags['P1'],
    mode='lines',
    name='Real Data',
    line=dict(color='black', width=2)
))

# Add predictions over time
for i in range(len(predictions)):
    forecast_index = df_p1_lags.index[i + 33 : i + 33 + 10]

    fig.add_trace(go.Scatter(
        x=forecast_index, 
        y=predictions[i],
        mode='lines',
        name=f'Prediction {i+1}',
        line=dict(dash='dash', width=1, color='blue')
    ))

    # Add confidence interval shading
    fig.add_trace(go.Scatter(
        x=list(forecast_index) + list(forecast_index[::-1]),
        y=list(upper_bounds[i]) + list(lower_bounds[i][::-1]),
        fill='toself',
        fillcolor='rgba(0,255,0,0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        name='Confidence Interval',
        showlegend=(i == 0)
    ))

# Customize layout
fig.update_layout(
    title="Walk-Forward Forecasting (10-month prediction horizon)",
    xaxis_title="Months",
    yaxis_title="P1 Values",
    legend_title="Legend",
    template="plotly"
)

fig.show()


NameError: name 'df_p1_lags' is not defined

Com macros

In [ ]:
from pmdarima import auto_arima
from pmdarima.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')


def choose_parameters_arima(product_id, df_train, target_col='Sales', winsorize=False, feature_set=None, horizon=10, seasonal=False, stationary=False):
    """
    Optimized function to prepare data, run auto_arima, and evaluate performance.
    """
    # Common Preprocessing
    X_train, X_val, y_train, y_val = prepare_time_series_data(df_train, product_id, target_col, feature_set, horizon, winsorize)

    # Convert to numpy arrays
    y_train, y_val = y_train.values, y_val.values
    X_train, X_val = X_train.values if not X_train.empty else None, X_val.values if not X_val.empty else None

    # Auto ARIMA Model
    model = auto_arima(y=y_train, X=X_train, stationary=stationary, seasonal=seasonal, suppress_warnings=True, stepwise=True, error_action='ignore', trace=False)

    # Walk-forward Validation
    predictions = []
    for i in range(len(y_val)):
        new_x = X_val[i].reshape(1, -1) if X_val is not None else None
        model.update([y_val[i]], X=new_x)
        pred = model.predict(n_periods=1, X=new_x)[0] if new_x is not None else model.predict(n_periods=1)[0]
        predictions.append(pred)

    # Evaluate Model
    metrics = calculate_metrics(y_val, predictions)

    return {
        'product_id': product_id,
        'winsorize': winsorize,
        'features': feature_set if feature_set else 'all',
        'best_params': {'order': model.order, 'seasonal_order': model.seasonal_order if seasonal else None, 'aic': model.aic(), 'bic': model.bic()},
        'metrics': metrics,
        'model': model}


def find_best_arima_config(product_id, df_train, target_col='Sales'):
    """
    Find optimal ARIMA configuration matching original testing approach
    """
    data = df_train[product_id].copy()
    available_features = [col for col in data.columns if col != target_col]
    
    # Generate feature combinations (max 3 for ARIMAX stability)
    feature_combinations = generate_feature_combinations(available_features, max_features=3)
    
    # Test all configurations - consistent with original approach
    results = []
    for winsorize in [True, False]:
        for features in feature_combinations:
            for seasonal in [False, True]:  # Test both seasonal and non-seasonal
                print(f"\nTesting config - Winsorize: {winsorize}, Features: {features}, Seasonal: {seasonal}")
                
                result = choose_parameters_arima(
                    product_id=product_id,
                    df_train=df_train,
                    target_col=target_col,
                    winsorize=winsorize,
                    feature_set=features,
                    seasonal=seasonal
                )
                results.append(result)
    
    # Find best configuration (lowest RMSE)
    best_config = min(results, key=lambda x: x['metrics']['RMSE'])

    print(f"\n✅ Best configuration for product {product_id}:")
    print(f"- Winsorize: {best_config['winsorize']}")
    print(f"- Features: {best_config['features']}")
    print(f"- Seasonal: {best_config['best_params']['seasonal_order'] is not None}")
    print(f"- Order (p,d,q): {best_config['best_params']['order']}")
    print(f"- RMSE: {best_config['metrics']['RMSE']:.2f}")
    print(f"- R²: {best_config['metrics']['R2']:.4f}")
    
    return best_config

def process_product_arima(product_id):
    """
    Consistent wrapper function matching original format
    """
    best_config = find_best_arima_config(
        product_id=product_id,
        df_train=df_train,
        target_col='Sales'
    )

    return {
        'Product_ID': product_id,
        'Model': 'ARIMA',
        'Winsorize': best_config['winsorize'],
        'Features': best_config['features'],
        'Order': best_config['best_params']['order'],
        'Seasonal_Order': best_config['best_params']['seasonal_order'],
        'RMSE': best_config['metrics']['RMSE'],
        'R²': best_config['metrics']['R2']
    }

#### 6. NeuralProphet

In [ ]:
from neuralprophet import NeuralProphet
import pandas as pd
from datetime import datetime
import logging
logging.getLogger('prophet').setLevel(logging.WARNING)
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)

def prepare_neuralprophet_data(df, target_col='Sales', features=None):
    """Convert dataframe to NeuralProphet format"""
    df = df.reset_index()
    if 'Date' not in df.columns and 'date' not in df.columns:
        df['ds'] = pd.date_range(start='2020-01-01', periods=len(df), freq='D')
    elif 'Date' in df.columns:
        df['ds'] = pd.to_datetime(df['Date'])
    elif 'date' in df.columns:
        df['ds'] = pd.to_datetime(df['date'])
    
    df['y'] = df[target_col]
    
    if features:
        for feature in features:
            if feature not in ['ds', 'y']:
                df[feature] = df[feature]
    
    return df[['ds', 'y'] + (features if features else [])]

def choose_parameters_neuralprophet(product_id, df_train, target_col='Sales', winsorize=False, feature_set=None, horizon=10, n_lags=10, n_forecasts=10, epochs=50):
    """
    Optimized NeuralProphet function with common preprocessing.
    """
    # Common Preprocessing
    X_train, X_val, y_train, y_val = prepare_time_series_data(df_train, product_id, target_col, feature_set, horizon, winsorize)

    # Convert to NeuralProphet format
    train_df = prepare_neuralprophet_data(pd.concat([y_train, X_train], axis=1), target_col, feature_set)
    val_df = prepare_neuralprophet_data(pd.concat([y_val, X_val], axis=1), target_col, feature_set)

    # Initialize NeuralProphet
    model = NeuralProphet(n_lags=n_lags, n_forecasts=n_forecasts, yearly_seasonality=False, weekly_seasonality=False, daily_seasonality=False, num_hidden_layers=2, d_hidden=32, learning_rate=0.01, epochs=epochs)

    # Add exogenous variables if present
    if feature_set:
        for feature in feature_set:
            model = model.add_lagged_regressor(feature)

    # Train model
    model.fit(train_df, freq='D', validation_df=val_df)

    # Make Predictions
    predictions = []
    for i in range(len(y_val)):
        new_data = train_df.append(val_df.iloc[:i])  # Expand training set with new validation points
        model.fit(new_data, freq='D')  # Retrain only when necessary

        future = model.make_future_dataframe(new_data, periods=1)
        forecast = model.predict(future)
        
        predictions.append(forecast['yhat1'].iloc[-1])  # Extract last prediction

    # Evaluate Model
    metrics = calculate_metrics(val_df['y'].values, predictions)

    return {
        'product_id': product_id,
        'winsorize': winsorize,
        'features': feature_set if feature_set else 'all',
        'best_params': {'n_lags': n_lags, 'n_forecasts': n_forecasts, 'epochs': epochs},
        'metrics': metrics,
        'model': model
    }

def find_best_neuralprophet_config(product_id, df_train, target_col='Sales'):
    """
    Find optimal NeuralProphet configuration matching original testing approach
    """
    data = df_train[product_id].copy()
    available_features = [col for col in data.columns if col != target_col]
    
    # Generate feature combinations
    feature_combinations = generate_feature_combinations(available_features, max_features=3)
    
    # Test configurations
    results = []
    for winsorize in [True, False]:
        for features in feature_combinations:
            print(f"\nTesting config - Winsorize: {winsorize}, Features: {features}")
            
            result = choose_parameters_neuralprophet(
                product_id=product_id,
                df_train=df_train,
                target_col=target_col,
                winsorize=winsorize,
                feature_set=features
            )
            results.append(result)
    
    # Find best configuration
    best_config = min(results, key=lambda x: x['metrics']['RMSE'])

    print(f"\n✅ Best NeuralProphet configuration for product {product_id}:")
    print(f"- Winsorize: {best_config['winsorize']}")
    print(f"- Features: {best_config['features']}")
    print(f"- RMSE: {best_config['metrics']['RMSE']:.2f}")
    print(f"- R²: {best_config['metrics']['R2']:.4f}")
    
    return best_config

def process_product_neuralprophet(product_id):
    """
    Consistent wrapper function matching original format
    """
    best_config = find_best_neuralprophet_config(
        product_id=product_id,
        df_train=df_train,
        target_col='Sales'
    )

    return {
        'Product_ID': product_id,
        'Model': 'NeuralProphet',
        'Winsorize': best_config['winsorize'],
        'Features': best_config['features'],
        'Parameters': best_config['best_params'],
        'RMSE': best_config['metrics']['RMSE'],
        'R²': best_config['metrics']['R2']}

#### Prediction for he Macro Features used

In [ ]:
# Define the function to classify variables
def classify_variable(series):
    """Classifies variable based on normality and stationarity tests."""
    
    # Remove NaN values for testing
    clean_series = series.dropna()

    # Check for normality
    if len(clean_series) > 3:
        stat, p_value = shapiro(clean_series)
        is_normal = p_value > 0.05  # p-value > 0.05 means normal
    else:
        is_normal = False  # Not enough data to test normality

    # Check for stationarity
    if len(clean_series) > 3:
        adf_stat, adf_p_value, _, _, _, _ = adfuller(clean_series)
        is_stationary = adf_p_value < 0.05  # p-value < 0.05 means stationary
    else:
        is_stationary = False  # Not enough data

    return is_normal, is_stationary

# Function to automatically fill missing values
def auto_impute_missing_values(df_train, df_test):
    """Automatically selects the best imputation method for each missing variable."""
    
    # Identify missing columns in test set
    missing_columns = df_test.columns[df_test.isnull().any()]
    
    # Iterate through missing columns
    for col in missing_columns:
        print(f"Processing: {col}")

        series = df_train[col]  # Use train data for imputation
        is_normal, is_stationary = classify_variable(series)

        if is_normal:
            # Case 1: Normally distributed → Sample from normal distribution
            print(f" - {col} is normal → Using Mean & Std Sampling")
            mean_value, std_value = series.mean(), series.std()
            num_missing = df_test[col].isnull().sum()
            predictions = norm.rvs(loc=mean_value, scale=std_value, size=num_missing)
        
        elif is_stationary:
            # Case 2: Stationary but non-normal → Simple Exponential Smoothing
            print(f" - {col} is stationary → Using Simple Exponential Smoothing")
            model = SimpleExpSmoothing(series.dropna()).fit()
            predictions = model.forecast(steps=df_test[col].isnull().sum())

        elif not is_stationary:
            # Case 3: Non-Stationary → ARIMA
            print(f" - {col} is non-stationary → Using ARIMA")
            model = ARIMA(series.dropna(), order=(1, 1, 1))  # (p,d,q) chosen based on domain knowledge
            fitted_model = model.fit()
            predictions = fitted_model.forecast(steps=df_test[col].isnull().sum())

        else:
            # Case 4: If nothing works → Use XGBoost Regression
            print(f" - {col} is complex → Using XGBoost Regression")
            train_data = df_train.dropna(subset=[col])  # Drop missing values for training
            X_train = train_data.drop(columns=[col])  # Exclude target column
            y_train = train_data[col]  # Target column

            X_test = df_test.loc[df_test[col].isnull(), X_train.columns]  # Only missing values

            model = XGBRegressor(n_estimators=100, learning_rate=0.1)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)

        # Assign predictions
        missing_indexes = df_test[df_test[col].isnull()].index
        df_test.loc[missing_indexes, col] = predictions

    return df_test

# Example usage
df_train = remerged_data[1]  # Use remerged train data
df_test = test_1.copy()  # Copy test set

# Apply automatic imputation
df_test_filled = auto_impute_missing_values(df_train, df_test)

# Check results
print(df_test_filled.head())